In [1]:
!pip3 install pandas

In [1]:
import yara
import os
from typing import List
from pathlib import Path
import shutil
import pandas as pd
from itertools import chain

In [2]:
rules_1_of_3 = '/home/pedro/Downloads/teste/Malware/rules/1-3_rules.yar'
rules_2_of_3 = '/home/pedro/Downloads/teste/Malware/rules/2-3_rules.yar'
rules_3_of_3 = '/home/pedro/Downloads/teste/Malware/rules/3-3_rules.yar'
rules_path = [rules_1_of_3, rules_2_of_3, rules_3_of_3]
rules = []
for rule_path in rules_path:
    rule = yara.compile(rule_path)
    rules.append(rule)

In [3]:
i = 1
for rule in rules:
    rule.save('/home/pedro/Documents/IME/2021/PFC/ofuscador1/compiled_' + str(i) + '_of3')
    i += 1

In [5]:
class Malware:
    def __init__(self, path: str, name: str, detected_rules: List[str]):
        self.path = path
        self.name = name
        self.detected_rules = detected_rules
    
    def cropp(self):
        max_begin = self._max_begin()
        min_end = self._min_end(max_begin)
        cropped = open(self.path + 'cropped/' + self.name + '.cropped', 'wb')
        cropped.write(min_end)
        cropped.close()
    
    def _max_begin(self):
        f = open(self.path + self.name, 'rb')
        f_content = f.read()
        begin = 0
        end = len(f_content)
        newbegin = (begin + end)//2
        infected_tempfile = f_content
        
        while(newbegin > begin):
            tempfile = f_content[newbegin: end]
            if len(tempfile) <= 2048:
                break
            if any (rule.match(data=tempfile) for rule in rules):
                begin = newbegin
                newbegin = (begin + end)//2
                infected_tempfile = tempfile
            else:
                newbegin = (begin + newbegin)//2
        f.close()
        return infected_tempfile

    def _min_end(self, file):
        f_content = file
        begin = 0
        end = len(f_content)
        newend = (begin + end)//2
        infected_tempfile = f_content

        while(abs(newend - end) > 1):
            tempfile = f_content[begin: newend]
            if len(tempfile) <= 2048:
                break
            if any (rule.match(data=tempfile) for rule in rules):
                end = newend
                newend = (begin + end)//2
                infected_tempfile = tempfile
            else:
                newend = (end + newend)//2
        if any(rule.match(data=infected_tempfile[:-1]) for rule in rules):
            return infected_tempfile[:-1]
        return infected_tempfile

In [6]:
path = '/home/pedro/Downloads/teste/Malware/minimize/'
malwares = []
if os.path.exists(os.path.join(path, 'cropped')):
    shutil.rmtree(os.path.join(path, 'cropped'))
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        file_path = os.path.join(path, filename)
        for rule in rules:
            detected_rules = rule.match(file_path)
            if len(detected_rules) > 0:
                malwares.append(Malware(path, filename, detected_rules))

In [7]:
len(malwares)
names = []
for malware in malwares:
    names.append(malware.name)

names = list(set(names))

In [8]:
len(names)

49

In [9]:
Path(path+"cropped").mkdir(parents=True, exist_ok=True)
for m in malwares:
    m.cropp()

In [10]:
path = '/home/pedro/Downloads/teste/Malware/minimize/cropped'
cropped = []
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        file_path = os.path.join(path, filename)
        detected_rules = [rule.match(file_path) for rule in rules]
        detected_rules = list(chain(*detected_rules))
        if len(detected_rules) > 0:
            cropped.append(Malware(path, filename, detected_rules))

In [11]:
for malware in cropped:
    print(malware.name)
len(cropped)

rdrbs100.dat.cropped
io.dat.cropped
downloader.dat.cropped
Filereg.dat.cropped
msrll.dat.cropped
kinject.dat.cropped
shellcode.dat.cropped
bdcli100.dat.cropped
123.dat.cropped
delta.dat.cropped
NL.dat.cropped
listdlls.dat.cropped
out.dat.cropped
fu.dat.cropped
unknown.dat.cropped
evilpdf-15PDF.dat.cropped
isi32.dat.cropped
bintext.dat.cropped
rksniffer.dat.cropped
withme.dat.cropped
getdown.dat.cropped
Hanuman.dat.cropped
brbbot.dat.cropped
agobot.dat.cropped
gy.dat.cropped
ghyte.dat.cropped
sc2.dat.cropped
windowsxp2.dat.cropped
lab1.dat.cropped
tnnbtib.dat.cropped
lab1-unpacked.dat.cropped
storm.dat.cropped
agup.dat.cropped
Windows Live Messenger.dat.cropped
extracted.dat.cropped
dsnx.dat.cropped
thoseWebForPDF.dat.cropped
xd.dat.cropped
hxdef100.dat.cropped
blinky.dat.cropped
server.dat.cropped
spybot.dat.cropped
srvcp.dat.cropped
9.dat.cropped
sdbot.dat.cropped
rep.dat.cropped
ouchPDF.dat.cropped
ysreef.dat.cropped
lab1-unpacked_SCY.dat.cropped


49

In [12]:
path

'/home/pedro/Downloads/teste/Malware/minimize/cropped'

In [13]:
cmd_path = '/home/pedro/Downloads/teste/Malware/minimize/cropped/MAGR.jar'
malware_path = '../../../../../Downloads/teste/Malware/minimize/cropped/'
copy_magr = 'cp ../../../../../Downloads/teste/Malware/minimize/MAGR.jar ' + path
os.system(copy_magr)

0

In [14]:
for malware in cropped:
    grammar = 'java -jar -ea ' + cmd_path + ' -iokab ' + malware_path + malware.name + ' 1> ' + malware_path + malware.name + '-iokab.txt 2>&1'
    os.system(grammar)
    r_grammar = 'java -jar -ea ' + cmd_path + ' -iokabr ' + malware_path + malware.name + ' 1> ' + malware_path + malware.name + '-iokabr.txt 2>&1'
    os.system(r_grammar)

In [15]:
path = '/home/pedro/Downloads/teste/Malware/minimize/cropped'

In [16]:
detected_after_grammar = []
binary_after_grammar = []

detected_after_r_grammar = []
binary_after_r_grammar = []

for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        if filename.endswith('cropped-iokab_binary_grammar.dat'):
            binary_after_grammar.append(filename)
            file_path = os.path.join(path, filename)
            detected_rules = [rule.match(file_path) for rule in rules]
            print(detected_rules)
            print(bool(detected_rules[2]))
            print(any(rule.match(file_path) for rule in rules))
            detected_rules = list(chain(*detected_rules))
            if len(detected_rules) > 0:
                detected_after_grammar.append(Malware(path, filename, detected_rules))
        if filename.endswith('.cropped-iokabr_binary_grammar.dat'):
            binary_after_r_grammar.append(filename)
            file_path = os.path.join(path, filename)
            detected_rules = [rule.match(file_path) for rule in rules]
            detected_rules = list(chain(*detected_rules))
            if len(detected_rules) > 0:
                detected_after_r_grammar.append(Malware(path, filename, detected_rules))

[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], [SEH__vba]]
True
True
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[invalid_trailer_structure, invalid_xref_numbers], [invalid_trailer_structure, invalid_xref_numbers], []]
False
True
[[], [], [Str_Win32_Wininet_Library]]
True
True
[[], [], [Str_Win32_Internet_API]]
True
True
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], []]
False
False
[[], [], [Str_Win32_Winsock2_Library]]
True
True
[[], [], 

In [17]:
detected_after_grammar[0].name

'extracted.dat.cropped-iokab_binary_grammar.dat'

In [18]:
len(binary_after_grammar)

48

In [19]:
len(detected_after_grammar)

7

In [20]:
len(binary_after_r_grammar)

48

In [21]:
len(detected_after_r_grammar)

4

In [22]:
table_data = {
    'name': [],
    'cropped_size': [],
    'grammar_size': [],
    'grammar_rules': [],
    'detected_grammar': [],
    'r_grammar_size': [],
    'r_grammar_rules': [],
    'detected_r_grammar': []
}
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        if filename.endswith('.cropped'):
            try:
               
                f = open(path + '/'+ filename + '-iokab_binary_grammar.dat', 'rb')
                table_data['grammar_size'].append(len(f.read()))
                
                detected_rules = [rule.match(path + '/'+ filename + '-iokab_binary_grammar.dat') for rule in rules]
                detected_rules = list(chain(*detected_rules))
                table_data['grammar_rules'].append(detected_rules)
                table_data['detected_grammar'].append(any(detected_rules))

                f = open(path + '/'+ filename + '-iokabr_binary_grammar.dat', 'rb')
                table_data['r_grammar_size'].append(len(f.read()))
                detected_rules = [rule.match(path + '/'+ filename + '-iokabr_binary_grammar.dat') for rule in rules]
                #data=f.read()
                #detected_rules = [rule.match(data=data) for rule in rules]
                
                detected_rules = list(chain(*detected_rules))
                table_data['r_grammar_rules'].append(detected_rules)
                table_data['detected_r_grammar'].append(any(detected_rules))

                
                table_data['name'].append(filename)
                f = open(path + '/'+ filename, 'rb')
                table_data['cropped_size'].append(len(f.read()))
            
            except:
                pass

In [23]:
len(table_data['name'])

48

In [24]:
df = pd.DataFrame(table_data)

In [25]:
df['%compression_grammar'] = (df['grammar_size']/df['cropped_size'] - 1)*100

In [26]:
df['%compression_r_grammar'] = (df['r_grammar_size']/df['cropped_size'] - 1)*100

In [27]:
cols = ['name', 'cropped_size', 'grammar_size', '%compression_grammar', 'grammar_rules', 'detected_grammar', 'r_grammar_size', '%compression_r_grammar', 'r_grammar_rules', 'detected_r_grammar']

In [28]:
df = df[cols]

In [32]:
df.head(49)

,name,cropped_size,grammar_size,%compression_grammar,grammar_rules,detected_grammar,r_grammar_size,%compression_r_grammar,r_grammar_rules,detected_r_grammar
0,rdrbs100.dat.cropped,2763,2760,-0.108578,[],False,2788,0.904814,[],False
1,io.dat.cropped,3280,2628,-19.878049,[],False,2652,-19.146341,[],False
2,downloader.dat.cropped,1023,830,-18.866080,[],False,868,-15.151515,[],False
3,Filereg.dat.cropped,2559,3395,32.669011,[],False,3459,35.169988,[],False
4,msrll.dat.cropped,2623,3636,38.619901,[],False,3683,40.411742,[],False
5,kinject.dat.cropped,3327,3449,3.666967,[],False,3448,3.636910,[],False
6,shellcode.dat.cropped,3327,600,-81.965735,[],False,600,-81.965735,[],False
7,bdcli100.dat.cropped,3327,3034,-8.806733,[],False,3041,-8.596333,[],False
8,123.dat.cropped,2327,3173,36.355823,[],False,3237,39.106145,[],False
9,delta.dat.cropped,3212,3421,6.506849,[],False,3407,6.070984,[],False


In [30]:
df.to_csv("ofuscation_table.csv", encoding="utf-8", index=False)

In [33]:
os.system('pwd')

0